In [ ]:
import requests
from dotenv import load_dotenv
import os 
import time
import pandas as pd
import numpy as np

### Hit graphQL endpoint and fetch all the data using course id

In [ ]:
from demo_main import get_course_data, save_to_excel

df = pd.read_csv('demo.csv')

processed_indices = []
count = 0
for index, row in df.iterrows():
    course_id = int(row['thinkific_id'])
    print(course_id)
    try:
        count += 1
        if count == 6:
            time.sleep(50)
            count = 0
        course_data = get_course_data(course_id)
        save_to_excel(course_data, "course_full_data.xlsx")
        print("Data saved to course_full_data.xlsx successfully!")
        processed_indices.append(index)
        df = df.drop(index)
        df.to_csv('demo.csv', index=False)
    except Exception as e:
        print(e)

print("Processing completed.")



### Append the users to Hive learning API

In [ ]:

load_dotenv()

api_key = os.getenv('api_key')

def user_data_insertion(first_name, last_name, email):
    url = "https://redteam2.hivelearning.com/api/beta/users"
    headers = {
        'Accept': 'application/json',
        'x-api-key': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        'firstName': first_name,
        'lastName': last_name,
        'email' : email
    }
    response = requests.post(url, json=payload,headers=headers) 
    return response.json()

df_user = pd.read_csv('thinkific_users.csv')
# df_course = pd.read_excel('course_data.xlsx')

for index, row in df_user.iterrows():
        id_response = user_data_insertion(row['first_name'], row['last_name'], row['email'])
        print("ID:", id_response)
        try :       
            user_id = id_response['data']['id']
            print("User ID:", user_id)
            df_user.at[index, 'user_id'] = user_id
            df_user.to_csv('thinkific_users.csv', index=False)
        except:    
            print("User ID not found")

### Append Groups to Hive learning API

In [ ]:

load_dotenv()

api_key = os.getenv('api_key')

def course_data_insertion(title, description, image):
    url = "https://redteam2.hivelearning.com/api/beta/groups"
    headers = {
        'Accept': 'application/json',
        'x-api-key': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        'title': title,
        'image': image,
        'privacy': 'closed',
        'owner_id': '29cd3a30-32f8-4ab6-b993-de2295c3a111'
    }
    if description:
        payload['description'] = description

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

df_course = pd.read_excel('course_full_data.xlsx')
df_course_unique = df_course.drop_duplicates(subset=['course_id'])

output_file = 'course_uuid.xlsx'
error_file = 'error_courses.xlsx'

if not os.path.isfile(output_file):
    pd.DataFrame(columns=['course_id', 'course_title', 'course_description', 'cardImage_url', 'course_uuid']).to_excel(output_file, index=False)

if not os.path.isfile(error_file):
    pd.DataFrame(columns=['course_id']).to_excel(error_file, index=False)

df_existing = pd.read_excel(output_file)
processed_ids = df_existing['course_id'].tolist()

# Read existing error data
df_error = pd.read_excel(error_file)
error_ids = df_error['course_id'].tolist()

df_course_unique = df_course_unique.replace([np.inf, -np.inf, np.nan], '')

for index, row in df_course_unique.iterrows():
    if row['course_id'] in processed_ids or row['course_id'] in error_ids:
        print(f"Skipping already processed or errored course_id: {row['course_id']}")
        continue

    try:
        course_response = course_data_insertion(row['course_title'], row.get('course_description'), row['cardImage_url'])
        print("Response:", course_response)

        if course_response and 'data' in course_response and 'id' in course_response['data']:
            course_uuid = course_response['data']['id']
            course_id = row['course_id']
            print("Course UUID:", course_uuid)

            result = {
                'course_id': course_id,
                'course_title': row['course_title'],
                'course_description': row.get('course_description'),
                'cardImage_url': row['cardImage_url'],
                'course_uuid': course_uuid
            }

            df_existing = pd.concat([df_existing, pd.DataFrame([result])], ignore_index=True)
            df_existing.to_excel(output_file, index=False)
            processed_ids.append(course_id)
        else:
            print(f"Unexpected response format for course_id {row['course_id']}: {course_response}")
            error_ids.append(row['course_id'])
            df_error = pd.concat([df_error, pd.DataFrame([{'course_id': row['course_id']}])], ignore_index=True)
            df_error.to_excel(error_file, index=False)

    except Exception as e:
        print(f"Error in processing course_id {row['course_id']}: {e}")
        error_ids.append(row['course_id'])
        df_error = pd.concat([df_error, pd.DataFrame([{'course_id': row['course_id']}])], ignore_index=True)
        df_error.to_excel(error_file, index=False)


### Append the pathways to Hive Leaning API

In [ ]:

# Load environment variables
load_dotenv()

api_key = os.getenv('api_key')

def pathway_data_insertion(title, group_id):
    url = "https://redteam2.hivelearning.com/api/beta/resources"
    headers = {
        'Accept': 'application/json',
        'x-api-key': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        'author_id': '29cd3a30-32f8-4ab6-b993-de2295c3a111', 
        'group_id': group_id,
        'title': title,
        'type': 'pathway'
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

error_file = 'error_pathways.xlsx'

if os.path.exists(error_file):
    df_error = pd.read_excel(error_file)
else:
    df_error = pd.DataFrame(columns=['course_id', 'chapter_id'])

df_course = pd.read_excel('merged_course_data_updated.xlsx')

if 'pathway_uuid' not in df_course.columns:
    df_course['pathway_uuid'] = np.nan

for index, row in df_course.iterrows():
    if not pd.isna(row['pathway_uuid']):
        print("Pathway UUID already exists for course_id:", row['chapter_id'])
        continue

    try:
        pathway_response = pathway_data_insertion(row['chapter_title'], row['course_uuid'])
        print("Response:", pathway_response)

        if pathway_response and 'data' in pathway_response and 'id' in pathway_response['data']:
            pathway_uuid = pathway_response['data']['id']
            print("Pathway UUID:", pathway_uuid)
            df_course.at[index, 'pathway_uuid'] = pathway_uuid  # Update DataFrame
            df_course.to_excel('merged_course_data_updated.xlsx', index=False)

        else:
            print(f"Unexpected response format for course_id {row['course_id']}: {pathway_response}")
            df_error = pd.concat([df_error, pd.DataFrame([{'course_id': row['course_id'], 'chapter_id': row['chapter_id']}])], ignore_index=True)
            df_error.to_excel(error_file, index=False)

    except Exception as e:
        print(f"Error processing course_id {row['course_id']}: {e}")
        df_error = pd.concat([df_error, pd.DataFrame([{'course_id': row['course_id'], 'chapter_id': row['chapter_id']}])], ignore_index=True)

        df_error.to_excel(error_file, index=False)
